<a href="https://colab.research.google.com/github/VishnuDattu/Generative-AI/blob/main/GEN_Course_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Conv2DTranspose, Input
import os
import cv2
import numpy as np

dataset_path = "/content/drive/MyDrive/dataset/animal1"
def load_images(path, img_size=(256, 256)):
    images = []
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 127.5 - 1  # Normalize to [-1, 1]
            images.append(img)
        else:
            print(f"Warning: Could not load image: {img_path}")
    return np.array(images)

# Define the generator model
def build_generator():
    model = tf.keras.Sequential([
        Input(shape=(100,)),
        tf.keras.layers.Dense(256 * 16 * 16, activation="relu"),
        tf.keras.layers.Reshape((16, 16, 256)),
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        Conv2DTranspose(32, (4, 4), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        Conv2DTranspose(16, (4, 4), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        Conv2D(3, (7, 7), activation="tanh", padding="same")
    ])
    return model

# Define the discriminator model
def build_discriminator(img_shape):
    model = tf.keras.Sequential([
        Input(shape=img_shape),
        Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        LeakyReLU(negative_slope=0.2),
        Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        BatchNormalization(),
        LeakyReLU(negative_slope=0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

# Compile the models
def compile_models(generator, discriminator):
    discriminator.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=["accuracy"])
    discriminator.trainable = False

    gan_input = tf.keras.layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = tf.keras.models.Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))

    return gan

# Train the GAN
def train_gan(gan, generator, discriminator, epochs, batch_size, real_images):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, real_images.shape[0], half_batch)
        real_imgs = real_images[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))
        generated_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)

        g_loss = gan.train_on_batch(noise, valid_y)

        # Print Progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc: {100 * d_loss[1]}] [G loss: {g_loss}]")

# Load images and train the GAN
real_images = load_images(dataset_path)
if len(real_images) == 0:
    raise ValueError("No images loaded. Please check the dataset path and ensure it contains valid images.")
img_shape = real_images.shape[1:]
generator = build_generator()
discriminator = build_discriminator(img_shape)
gan = compile_models(generator, discriminator)
train_gan(gan, generator, discriminator, epochs=100, batch_size=64, real_images=real_images)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
0 [D loss: 0.6996440887451172, acc: 45.3125] [G loss: 0.7137466073036194]
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2